**Title-Bank Customer Churn Model**

Objective- to predict the churn of bank customer means whether the customer will leave the bank or not

Data Sourse- for this we will use the data set of ybi foundation's githug page data set

Import Libraries



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as ans

Import Data


In [2]:
df= pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Bank%20Churn%20Modelling.csv')

In [3]:
df.sample(5)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,Num Of Products,Has Credit Card,Is Active Member,Estimated Salary,Churn
8564,15584704,Chiazagomekpele,519,France,Male,48,10,71083.98,1,1,0,137959.00,0
342,15681081,Marrero,545,Spain,Female,47,5,0.00,2,1,1,38970.14,0
5208,15664737,Lei,779,Spain,Female,38,7,0.00,2,1,1,138542.87,0
1567,15683124,Evans,713,France,Male,53,6,115029.40,1,0,0,191521.32,1
8522,15619892,Page,644,Spain,Male,18,8,0.00,2,1,0,59172.42,0


In [4]:
df=df.set_index('CustomerId')

In [5]:
df.describe()

,CreditScore,Age,Tenure,Balance,Num Of Products,Has Credit Card,Is Active Member,Estimated Salary,Churn
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 15634602 to 15628319
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Surname           10000 non-null  object 
 1   CreditScore       10000 non-null  int64  
 2   Geography         10000 non-null  object 
 3   Gender            10000 non-null  object 
 4   Age               10000 non-null  int64  
 5   Tenure            10000 non-null  int64  
 6   Balance           10000 non-null  float64
 7   Num Of Products   10000 non-null  int64  
 8   Has Credit Card   10000 non-null  int64  
 9   Is Active Member  10000 non-null  int64  
 10  Estimated Salary  10000 non-null  float64
 11  Churn             10000 non-null  int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 1015.6+ KB


so data has no null values with all the details of coustmer and he did leave the bank or not

In [7]:
df['Num Of Products'].value_counts()

Num Of Products
1    5084
2    4590
3     266
4      60
Name: count, dtype: int64

now we have to encode the categorical and numerical columns

In [8]:
df.replace({'Num Of Products':{1:0,2:1,3:1,4:1}},inplace=True)

In [9]:
df['Has Credit Card'].value_counts()

Has Credit Card
1    7055
0    2945
Name: count, dtype: int64

In [10]:
df['Is Active Member'].value_counts()

Is Active Member
1    5151
0    4849
Name: count, dtype: int64

In [11]:
df.loc[df['Balance']==0,'Churn'].value_counts()

Churn
0    3117
1     500
Name: count, dtype: int64

In [12]:
#now people having zero bankbalance has high chance of leaving the bank so creat a new column with name of zero bank balance


In [13]:
df["Zero Balance"]=np.where(df['Balance']>0,1,0)

**train test split**

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x=df.drop(['Surname','Churn'],axis=1)
y=df['Churn']

In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [17]:
#now like the churn column have less values with 1 so we have to match it coz its not good
#there are two ways to do it wether the over sampler or under sampler well use over sampler coz itll not lose any info

In [18]:
from imblearn.over_sampling import RandomOverSampler

In [19]:
ros=RandomOverSampler()

In [20]:
x_ros,y_ros=ros.fit_resample(x,y)

In [21]:
y_ros.value_counts()

Churn
1    7963
0    7963
Name: count, dtype: int64

In [22]:

x_train_ros,x_test_ros,y_train_ros,y_test_ros=train_test_split(x_ros,y_ros,test_size=0.2)

we will use column transformer to transform our nominam and ordinal categorical data and to scale numerical data

In [23]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [24]:
oe=OrdinalEncoder()
ohe=OneHotEncoder()
ss=StandardScaler()

In [25]:
transformer= ColumnTransformer([('tnf1',ohe,['Geography','Gender']),('tnf2',ss,['CreditScore','Age','Tenure','Balance','Estimated Salary'])],remainder='passthrough')

In [26]:
x_train_trans=transformer.fit_transform(x_train)
x_test_trans=transformer.transform(x_test)

In [27]:
x_train_ros_trans=transformer.fit_transform(x_train_ros)
x_test_ros_trans=transformer.transform(x_test_ros)

support vecotor classifier

In [28]:
from sklearn.svm import SVC

In [29]:
svc=SVC()

In [30]:
svc.fit(x_train_trans,y_train)
svc.fit(x_train_ros_trans,y_train_ros)

SVC()

In [31]:
y_pred=svc.predict(x_test_trans)
y_pred_ros=svc.predict(x_test_ros_trans)

time to check the accuracy score

In [34]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

In [36]:
accuracy_score(y_test,y_pred)

0.757

In [54]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.75      0.83      1595
           1       0.44      0.80      0.57       405

    accuracy                           0.76      2000
   macro avg       0.69      0.77      0.70      2000
weighted avg       0.84      0.76      0.78      2000



In [37]:
accuracy_score(y_test_ros,y_pred_ros)

0.7639673571876961

In [38]:
confusion_matrix(y_test_ros,y_pred_ros)

array([[1199,  349],
       [ 403, 1235]])

In [39]:
print(classification_report(y_test_ros,y_pred_ros))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76      1548
           1       0.78      0.75      0.77      1638

    accuracy                           0.76      3186
   macro avg       0.76      0.76      0.76      3186
weighted avg       0.76      0.76      0.76      3186



even tho the accuracy is aound 75 but we can do better

In [40]:
from sklearn.model_selection import GridSearchCV

In [55]:
param_grid={'C':[0.1,1,10],'gamma':[1,0.1,0.01],'kernel':['rbf'],'class_weight':['balanced']}

In [57]:
best_model=GridSearchCV(SVC(),param_grid,refit=True,verbose=2,cv=2)
best_model.fit(x_train_trans,y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END ..C=0.1, class_weight=balanced, gamma=1, kernel=rbf; total time=   2.7s
[CV] END ..C=0.1, class_weight=balanced, gamma=1, kernel=rbf; total time=   3.4s
[CV] END C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   1.5s
[CV] END C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   1.4s
[CV] END C=0.1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   1.6s
[CV] END C=0.1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   1.7s
[CV] END ....C=1, class_weight=balanced, gamma=1, kernel=rbf; total time=   1.8s
[CV] END ....C=1, class_weight=balanced, gamma=1, kernel=rbf; total time=   1.8s
[CV] END ..C=1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   2.0s
[CV] END ..C=1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   2.1s
[CV] END .C=1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   1.4s
[CV] END .C=1, class_weight=balanced, gamma=0.0

GridSearchCV(cv=2, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01], 'kernel': ['rbf']},
             verbose=2)

In [59]:
print(best_model.best_estimator_)

SVC(C=10, class_weight='balanced', gamma=1)


In [61]:
confusion_matrix(y_test,best_model.predict(x_test_trans))

array([[1545,   50],
       [  11,  394]])

In [63]:
print(classification_report(y_test,best_model.predict(x_test_trans)))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1595
           1       0.89      0.97      0.93       405

    accuracy                           0.97      2000
   macro avg       0.94      0.97      0.95      2000
weighted avg       0.97      0.97      0.97      2000



In [64]:
accuracy_score(y_test,best_model.predict(x_test_trans))

0.9695

In [67]:
best_model=GridSearchCV(SVC(),param_grid,refit=True,verbose=2,cv=2)
best_model.fit(x_train_ros_trans,y_train_ros)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END ..C=0.1, class_weight=balanced, gamma=1, kernel=rbf; total time=   6.6s
[CV] END ..C=0.1, class_weight=balanced, gamma=1, kernel=rbf; total time=   5.4s
[CV] END C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   4.6s
[CV] END C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   4.3s
[CV] END C=0.1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   4.1s
[CV] END C=0.1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   5.0s
[CV] END ....C=1, class_weight=balanced, gamma=1, kernel=rbf; total time=   5.0s
[CV] END ....C=1, class_weight=balanced, gamma=1, kernel=rbf; total time=   4.3s
[CV] END ..C=1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   4.1s
[CV] END ..C=1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   4.0s
[CV] END .C=1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   3.6s
[CV] END .C=1, class_weight=balanced, gamma=0.0

GridSearchCV(cv=2, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01], 'kernel': ['rbf']},
             verbose=2)

In [68]:
print(best_model.best_estimator_)

SVC(C=10, class_weight='balanced', gamma=1)


In [69]:
confusion_matrix(y_test_ros,best_model.predict(x_test_ros_trans))

array([[1342,  206],
       [  46, 1592]])

In [70]:
print(classification_report(y_test_ros,best_model.predict(x_test_ros_trans)))

              precision    recall  f1-score   support

           0       0.97      0.87      0.91      1548
           1       0.89      0.97      0.93      1638

    accuracy                           0.92      3186
   macro avg       0.93      0.92      0.92      3186
weighted avg       0.92      0.92      0.92      3186



In [71]:
accuracy_score(y_test_ros,best_model.predict(x_test_ros_trans))

0.9209039548022598

so our this model without using ros can predict the churn of a bank customer with around 96% of accuracy and with 97% recall mean it can predict the churn with 97% accuracy.